# 4. szekció – SLM és LLM összehasonlítása

Hasonlítsuk össze a késleltetést és a válaszminőséget egy Kis Nyelvi Modell és egy nagyobb modell között, amely a Foundry Local segítségével fut.


## ⚡ Gyors kezdés

**Memória-optimalizált beállítás (Frissítve):**
1. A modellek automatikusan kiválasztják a CPU változatokat (bármilyen hardveren működik)
2. `qwen2.5-3b` használata a 7B helyett (kb. 4 GB RAM megtakarítás)
3. Automatikus portérzékelés (nincs szükség kézi konfigurációra)
4. Szükséges teljes RAM: ~8 GB ajánlott (modellek + operációs rendszer)

**Terminál beállítás (30 másodperc):**
```bash
foundry service start
foundry model run phi-4-mini
foundry model run qwen2.5-3b
```

Ezután futtasd ezt a notebookot! 🚀


### Magyarázat: Függőségek telepítése
Telepíti a minimális csomagokat (`foundry-local-sdk`, `openai`, `numpy`), amelyek szükségesek az időzítéshez és a csevegési kérésekhez. Biztonságosan újrafuttatható idempotens módon.


# Forgatókönyv
Hasonlíts össze egy reprezentatív Kis Nyelvi Modellt (SLM) egy nagyobb modellel egyetlen prompt alapján, hogy bemutathasd a kompromisszumokat:
- **Késleltetés különbsége** (valós idő másodpercben)
- **Tokenhasználat** (ha elérhető) mint az áteresztőképesség mutatója
- **Mintakimenet minősége** gyors szemrevételezéshez
- **Gyorsulás számítása** a teljesítményjavulás számszerűsítéséhez

**Környezeti változók:**
- `SLM_ALIAS` - Kis nyelvi modell (alapértelmezett: phi-4-mini, ~4GB RAM)
- `LLM_ALIAS` - Nagyobb nyelvi modell (alapértelmezett: qwen2.5-7b, ~7GB RAM)
- `COMPARE_PROMPT` - Teszt prompt az összehasonlításhoz
- `COMPARE_RETRIES` - Újrapróbálkozások a rugalmasság érdekében (alapértelmezett: 2)
- `FOUNDRY_LOCAL_ENDPOINT` - Szolgáltatási végpont felülírása (automatikusan észlelve, ha nincs beállítva)

**Hogyan működik (Hivatalos SDK minta):**
1. **FoundryLocalManager** inicializálja és kezeli a Foundry Local szolgáltatást
2. A szolgáltatás automatikusan elindul, ha nem fut (nincs szükség kézi beállításra)
3. A modellek aliasokból automatikusan konkrét azonosítókra kerülnek feloldásra
4. Hardver-optimalizált változatok kiválasztása (CUDA, NPU vagy CPU)
5. OpenAI-kompatibilis kliens végzi a chat-kiegészítéseket
6. Metrikák rögzítése: késleltetés, tokenek, kimeneti minőség
7. Az eredmények összehasonlítása a gyorsulási arány kiszámításához

Ez a mikro-összehasonlítás segít eldönteni, mikor indokolt egy nagyobb modell használata az adott felhasználási esetben.

**SDK Referencia:** 
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: A hivatalos mintát használja ../samples/workshop_utils.py

**Fő előnyök:**
- ✅ Automatikus szolgáltatásfelismerés és inicializálás
- ✅ Automatikus szolgáltatásindítás, ha nem fut
- ✅ Beépített modellfeloldás és gyorsítótárazás
- ✅ Hardver-optimalizáció (CUDA/NPU/CPU)
- ✅ OpenAI SDK kompatibilitás
- ✅ Robusztus hibakezelés újrapróbálkozásokkal
- ✅ Helyi következtetés (nincs szükség felhő API-ra)


## 🚨 Előfeltételek: Foundry Localnak futnia kell!

**Mielőtt futtatnád ezt a notebookot**, győződj meg róla, hogy a Foundry Local szolgáltatás be van állítva:

### Gyorsindítási parancsok (Futtasd a terminálban):

```bash
# 1. Start the Foundry Local service
foundry service start

# 2. Load the default models used in this comparison (CPU-optimized)
foundry model run phi-4-mini
foundry model run qwen2.5-3b

# 3. Verify models are loaded
foundry model ls

# 4. Check service health
foundry service status
```

### Alternatív modellek (ha az alapértelmezettek nem érhetők el):

```bash
# Even smaller alternatives (if memory is very limited)
foundry model run phi-3.5-mini
foundry model run qwen2.5-0.5b

# Or update the environment variables in this notebook:
# SLM_ALIAS = 'phi-3.5-mini'
# LLM_ALIAS = 'qwen2.5-1.5b'  # Or qwen2.5-0.5b for minimum memory
```

⚠️ **Ha kihagyod ezeket a lépéseket**, `APIConnectionError` hibát fogsz látni, amikor az alábbi notebook cellákat futtatod.


In [29]:
# Install dependencies
!pip install -q foundry-local-sdk openai numpy requests

### Magyarázat: Alapvető importok
Időzítési segédprogramokat, valamint Foundry Local / OpenAI klienseket hoz be, amelyeket modellinformációk lekérésére és csevegési kiegészítések végrehajtására használnak.


In [30]:
import os, time, json
from foundry_local import FoundryLocalManager
from openai import OpenAI
import sys
sys.path.append('../samples')
from workshop_utils import get_client, chat_once

### Magyarázat: Aliasok és Prompt Beállítás
Környezetfüggő aliasokat határoz meg kisebb és nagyobb modellekhez, valamint egy összehasonlító promptot. Állítsa be a környezeti változókat, hogy különböző modellcsaládokkal vagy feladatokkal kísérletezhessen.


In [31]:
# Default to CPU models for better memory efficiency
SLM = os.getenv('SLM_ALIAS', 'phi-4-mini')  # Auto-selects CPU variant
LLM = os.getenv('LLM_ALIAS', 'qwen2.5-3b')  # Smaller LLM, more memory-friendly
PROMPT = os.getenv('COMPARE_PROMPT', 'List 5 benefits of local AI inference.')
# Endpoint is now managed by FoundryLocalManager - it auto-detects or can be overridden
ENDPOINT = os.getenv('FOUNDRY_LOCAL_ENDPOINT', None)

### 💡 Memória-optimalizált konfiguráció

**Ez a jegyzetfüzet alapértelmezés szerint memóriahatékony modelleket használ:**
- `phi-4-mini` → ~4GB RAM (a Foundry Local automatikusan a CPU változatot választja)
- `qwen2.5-3b` → ~3GB RAM (a 7B helyett, amely ~7GB+ RAM-ot igényel)

**Port automatikus felismerése:**
- A Foundry Local különböző portokat használhat (általában 55769 vagy 59959)
- Az alábbi diagnosztikai cella automatikusan felismeri a megfelelő portot
- Nincs szükség kézi konfigurációra!

**Ha kevés RAM-mal (<8GB) rendelkezik, használjon még kisebb modelleket:**
```python
SLM = 'phi-3.5-mini'      # ~2GB
LLM = 'qwen2.5-0.5b'      # ~500MB
```


In [32]:
# Display current configuration
print("="*60)
print("CURRENT CONFIGURATION")
print("="*60)
print(f"SLM Model:     {SLM}")
print(f"LLM Model:     {LLM}")
print(f"SDK Pattern:   FoundryLocalManager (official)")
print(f"Endpoint:      {ENDPOINT or 'Auto-detect'}")
print(f"Test Prompt:   {PROMPT[:50]}...")
print(f"Retry Count:   2")
print("="*60)
print("\n💡 Using official Foundry SDK pattern from workshop_utils")
print("   → FoundryLocalManager handles service lifecycle")
print("   → Automatic model resolution and hardware optimization")
print("   → OpenAI-compatible API for inference")

CURRENT CONFIGURATION
SLM Model:     phi-4-mini
LLM Model:     qwen2.5-7b
SDK Pattern:   FoundryLocalManager (official)
Endpoint:      Auto-detect
Test Prompt:   List 5 benefits of local AI inference....
Retry Count:   2

💡 Using official Foundry SDK pattern from workshop_utils
   → FoundryLocalManager handles service lifecycle
   → Automatic model resolution and hardware optimization
   → OpenAI-compatible API for inference


### Magyarázat: Végrehajtási Segédeszközök (Foundry SDK Minta)
Az hivatalos Foundry Local SDK mintát használja, ahogyan azt a Workshop példák dokumentálják:

**Megközelítés:**
- **FoundryLocalManager** - Inicializálja és kezeli a Foundry Local szolgáltatást
- **Automatikus Érzékelés** - Automatikusan felismeri a végpontot és kezeli a szolgáltatás életciklusát
- **Modell Feloldás** - Aliasokat teljes modellazonosítókra old fel (pl. phi-4-mini → phi-4-mini-instruct-cpu)
- **Hardver Optimalizálás** - Kiválasztja a legjobb változatot az elérhető hardverhez (CUDA, NPU vagy CPU)
- **OpenAI Ügyfél** - A menedzser végpontjával konfigurálva OpenAI-kompatibilis API hozzáféréshez

**Ellenállóképességi Funkciók:**
- Exponenciális visszaállási próbálkozási logika (környezeten keresztül konfigurálható)
- Automatikus szolgáltatásindítás, ha nem fut
- Kapcsolat ellenőrzése inicializálás után
- Kíméletes hibakezelés részletes hibajelentéssel
- Modell gyorsítótárazás az ismételt inicializálás elkerülése érdekében

**Eredmény Struktúra:**
- Késleltetés mérése (valós idő)
- Tokenhasználat nyomon követése (ha elérhető)
- Mintakimenet (olvashatóság érdekében rövidítve)
- Hibák részletei sikertelen kérések esetén

Ez a minta a workshop_utils modult használja, amely követi a hivatalos SDK mintát.

**SDK Referencia:**
- Fő Repo: https://github.com/microsoft/Foundry-Local
- Python SDK: https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local
- Workshop Utils: ../samples/workshop_utils.py


In [39]:
def setup(alias: str, endpoint: str = None, retries: int = 3):
    """
    Initialize a Foundry Local model connection using official SDK pattern.
    
    This follows the workshop_utils pattern which uses FoundryLocalManager
    to properly initialize the Foundry Local service and resolve models.
    
    Args:
        alias: Model alias (e.g., 'phi-4-mini', 'qwen2.5-3b')
        endpoint: Optional endpoint override (usually auto-detected)
        retries: Number of connection attempts (default: 3)
    
    Returns:
        tuple: (manager, client, model_id, metadata) or (None, None, alias, error_metadata) if failed
    """
    import time
    
    last_err = None
    current_delay = 2  # seconds
    
    for attempt in range(1, retries + 1):
        try:
            print(f"[Init] Connecting to '{alias}' (attempt {attempt}/{retries})...")
            
            # Use the workshop utility which follows the official SDK pattern
            manager, client, model_id = get_client(alias, endpoint=endpoint)
            
            print(f"[OK] Connected to '{alias}' -> {model_id}")
            print(f"     Endpoint: {manager.endpoint}")
            
            return manager, client, model_id, {
                'endpoint': manager.endpoint,
                'resolved': model_id,
                'attempts': attempt,
                'status': 'success'
            }
            
        except Exception as e:
            last_err = e
            error_msg = str(e)
            
            # Provide helpful error messages
            if "Connection error" in error_msg or "connection refused" in error_msg.lower():
                print(f"[ERROR] Cannot connect to Foundry Local service")
                print(f"        → Is the service running? Try: foundry service start")
                print(f"        → Is the model loaded? Try: foundry model run {alias}")
            elif "not found" in error_msg.lower():
                print(f"[ERROR] Model '{alias}' not found in catalog")
                print(f"        → Available models: Run 'foundry model ls' in terminal")
                print(f"        → Download model: Run 'foundry model download {alias}'")
            else:
                print(f"[ERROR] Setup failed: {type(e).__name__}: {error_msg}")
            
            if attempt < retries:
                print(f"[Retry] Waiting {current_delay:.1f}s before retry...")
                time.sleep(current_delay)
                current_delay *= 2  # Exponential backoff
    
    # All retries failed - provide actionable guidance
    print(f"\n❌ Failed to initialize '{alias}' after {retries} attempts")
    print(f"   Last error: {type(last_err).__name__}: {str(last_err)}")
    print(f"\n💡 Troubleshooting steps:")
    print(f"   1. Ensure Foundry Local service is running:")
    print(f"      → foundry service status")
    print(f"      → foundry service start (if not running)")
    print(f"   2. Ensure model is loaded:")
    print(f"      → foundry model run {alias}")
    print(f"   3. Check available models:")
    print(f"      → foundry model ls")
    print(f"   4. Try alternative models if '{alias}' isn't available")
    
    return None, None, alias, {
        'error': f"{type(last_err).__name__}: {str(last_err)}",
        'endpoint': endpoint or 'auto-detect',
        'attempts': retries,
        'status': 'failed'
    }


def run(client, model_id: str, prompt: str, max_tokens: int = 180, temperature: float = 0.5):
    """
    Run inference with the configured model using OpenAI SDK.
    
    Args:
        client: OpenAI client instance (configured for Foundry Local)
        model_id: Model identifier (resolved from alias)
        prompt: Input prompt
        max_tokens: Maximum response tokens (default: 180)
        temperature: Sampling temperature (default: 0.5)
    
    Returns:
        dict: Response with timing, tokens, and content
    """
    import time
    
    start = time.time()
    
    try:
        response = client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            temperature=temperature
        )
        
        elapsed = time.time() - start
        
        # Extract response details
        content = response.choices[0].message.content
        
        # Try to extract token usage from multiple possible locations
        usage_info = {}
        if hasattr(response, 'usage') and response.usage:
            usage_info['prompt_tokens'] = getattr(response.usage, 'prompt_tokens', None)
            usage_info['completion_tokens'] = getattr(response.usage, 'completion_tokens', None)
            usage_info['total_tokens'] = getattr(response.usage, 'total_tokens', None)
        
        # Calculate approximate token count if API doesn't provide it
        # Rough estimate: ~4 characters per token for English text
        if not usage_info.get('total_tokens'):
            estimated_prompt_tokens = len(prompt) // 4
            estimated_completion_tokens = len(content) // 4
            estimated_total = estimated_prompt_tokens + estimated_completion_tokens
            usage_info['estimated_tokens'] = estimated_total
            usage_info['estimated_prompt_tokens'] = estimated_prompt_tokens
            usage_info['estimated_completion_tokens'] = estimated_completion_tokens
        
        return {
            'status': 'success',
            'content': content,
            'elapsed_sec': elapsed,
            'tokens': usage_info.get('total_tokens') or usage_info.get('estimated_tokens'),
            'usage': usage_info,
            'model': model_id
        }
        
    except Exception as e:
        elapsed = time.time() - start
        return {
            'status': 'error',
            'error': f"{type(e).__name__}: {str(e)}",
            'elapsed_sec': elapsed,
            'model': model_id
        }


print("✅ Execution helpers defined: setup(), run()")
print("   → Uses workshop_utils for proper SDK integration")
print("   → setup() initializes with FoundryLocalManager")
print("   → run() executes inference via OpenAI-compatible API")
print("   → Token counting: Uses API data or estimates if unavailable")

✅ Execution helpers defined: setup(), run()
   → Uses workshop_utils for proper SDK integration
   → setup() initializes with FoundryLocalManager
   → run() executes inference via OpenAI-compatible API
   → Token counting: Uses API data or estimates if unavailable


### Magyarázat: Előzetes önellenőrzés
Egy könnyű kapcsolódási tesztet futtat a FoundryLocalManager segítségével mindkét modell esetében. Ez ellenőrzi:
- A szolgáltatás elérhetőségét
- A modellek inicializálhatóságát
- Az aliasok tényleges modellazonosítókra történő feloldását
- A kapcsolat stabilitását az összehasonlítás futtatása előtt

A setup() függvény a workshop_utils hivatalos SDK mintáját használja.


In [34]:
# Simplified diagnostic: Just verify service is accessible
import requests

def check_foundry_service():
    """Quick diagnostic to verify Foundry Local is running."""
    # Try common ports
    endpoints_to_try = [
        "http://localhost:59959",
        "http://127.0.0.1:59959", 
        "http://localhost:55769",
        "http://127.0.0.1:55769",
    ]
    
    print("[Diagnostic] Checking Foundry Local service...")
    
    for endpoint in endpoints_to_try:
        try:
            response = requests.get(f"{endpoint}/health", timeout=2)
            if response.status_code == 200:
                print(f"✅ Service is running at {endpoint}")
                
                # Try to list models
                try:
                    models_response = requests.get(f"{endpoint}/v1/models", timeout=2)
                    if models_response.status_code == 200:
                        models_data = models_response.json()
                        model_count = len(models_data.get('data', []))
                        print(f"✅ Found {model_count} models available")
                        if model_count > 0:
                            print("   Models:", [m.get('id', 'unknown') for m in models_data.get('data', [])[:5]])
                except Exception as e:
                    print(f"⚠️  Could not list models: {e}")
                
                return endpoint
        except requests.exceptions.ConnectionError:
            continue
        except Exception as e:
            print(f"⚠️  Error checking {endpoint}: {e}")
    
    print("\n❌ Foundry Local service not found!")
    print("\n💡 To fix this:")
    print("   1. Open a terminal")
    print("   2. Run: foundry service start")
    print("   3. Run: foundry model run phi-4-mini")
    print("   4. Run: foundry model run qwen2.5-3b")
    print("   5. Re-run this notebook")
    return None

# Run diagnostic
discovered_endpoint = check_foundry_service()

if discovered_endpoint:
    print(f"\n✅ Service detected (will be managed by FoundryLocalManager)")
else:
    print(f"\n⚠️  No service detected - FoundryLocalManager will attempt to start it")

[Diagnostic] Checking Foundry Local service...

❌ Foundry Local service not found!

💡 To fix this:
   1. Open a terminal
   2. Run: foundry service start
   3. Run: foundry model run phi-4-mini
   4. Run: foundry model run qwen2.5-3b
   5. Re-run this notebook

⚠️  No service detected - FoundryLocalManager will attempt to start it


In [35]:
# Quick Fix: Start service and load models from notebook
# Uncomment the commands you need:

# !foundry service start
# !foundry model run phi-4-mini
# !foundry model run qwen2.5-3b
# !foundry model ls

print("⚠️  The commands above are commented out.")
print("Uncomment them if you want to start the service from the notebook.")
print("")
print("💡 Recommended: Run these commands in a separate terminal instead:")
print("   foundry service start")
print("   foundry model run phi-4-mini")
print("   foundry model run qwen2.5-3b")

⚠️  The commands above are commented out.
Uncomment them if you want to start the service from the notebook.

💡 Recommended: Run these commands in a separate terminal instead:
   foundry service start
   foundry model run phi-4-mini
   foundry model run qwen2.5-3b


### 🛠️ Gyors javítás: Foundry helyi indítása notebookból (opcionális)

Ha a fenti diagnosztika azt mutatja, hogy a szolgáltatás nem fut, megpróbálhatja innen elindítani:

**Megjegyzés:** Ez leginkább Windows rendszeren működik. Más platformokon használjon terminálparancsokat.


### ⚠️ Kapcsolati hibák elhárítása

Ha `APIConnectionError` hibaüzenetet kap, lehet, hogy a Foundry Local szolgáltatás nem fut, vagy a modellek nincsenek betöltve. Próbálja ki az alábbi lépéseket:

**1. Ellenőrizze a szolgáltatás állapotát:**
```bash
# In a terminal (not in notebook):
foundry service status
```

**2. Indítsa el a szolgáltatást (ha nem fut):**
```bash
foundry service start
```

**3. Töltse be a szükséges modelleket:**
```bash
# Load the models needed for comparison
foundry model run phi-4-mini
foundry model run qwen2.5-7b

# Or use alternative models:
foundry model run phi-3.5-mini
foundry model run qwen2.5-3b
```

**4. Ellenőrizze, hogy a modellek elérhetők-e:**
```bash
foundry model ls
```

**Gyakori problémák:**
- ❌ A szolgáltatás nem fut → Futtassa: `foundry service start`
- ❌ A modellek nincsenek betöltve → Futtassa: `foundry model run <model-name>`
- ❌ Portütközések → Ellenőrizze, hogy más szolgáltatás használja-e a portot
- ❌ Tűzfal blokkol → Győződjön meg róla, hogy a helyi kapcsolatok engedélyezve vannak

**Gyors megoldás:** Futtassa az alábbi diagnosztikai cellát a repülés előtti ellenőrzés előtt.


In [36]:
preflight = {}
retries = 2  # Number of retry attempts

for a in (SLM, LLM):
    mgr, c, mid, info = setup(a, endpoint=ENDPOINT, retries=retries)
    # Keep the original status from info (either 'success' or 'failed')
    preflight[a] = info

print('\n[Pre-flight Check]')
for alias, details in preflight.items():
    status_icon = '✅' if details['status'] == 'success' else '❌'
    print(f"  {status_icon} {alias}: {details['status']} - {details.get('resolved', details.get('error', 'unknown'))}")

preflight

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1

[Pre-flight Check]
  ✅ phi-4-mini: success - Phi-4-mini-instruct-cuda-gpu:4
  ✅ qwen2.5-7b: success - qwen2.5-7b-instruct-cuda-gpu:3


{'phi-4-mini': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'Phi-4-mini-instruct-cuda-gpu:4',
  'attempts': 1,
  'status': 'success'},
 'qwen2.5-7b': {'endpoint': 'http://127.0.0.1:59959/v1',
  'resolved': 'qwen2.5-7b-instruct-cuda-gpu:3',
  'attempts': 1,
  'status': 'success'}}

### ✅ Előzetes ellenőrzés: Modell elérhetőség

Ez a cella ellenőrzi, hogy mindkét modell elérhető-e a beállított végponton, mielőtt a összehasonlítást lefuttatná.


### Magyarázat: Összehasonlítás futtatása és eredmények gyűjtése
Mindkét aliason végigmegy a hivatalos Foundry SDK mintát követve:
1. Minden modellt inicializál a setup() segítségével (FoundryLocalManager-t használ)
2. Inference futtatása az OpenAI-kompatibilis API-val
3. Késleltetés, tokenek és mintakimenet rögzítése
4. JSON összefoglaló készítése összehasonlító elemzéssel

Ez ugyanazt a mintát követi, mint a Workshop példák a session04/model_compare.py fájlban.


In [40]:
results = []
retries = 2  # Number of retry attempts

for alias in (SLM, LLM):
    mgr, client, mid, info = setup(alias, endpoint=ENDPOINT, retries=retries)
    if client:
        r = run(client, mid, PROMPT)
        results.append({'alias': alias, **r})
    else:
        # If setup failed, record error
        results.append({
            'alias': alias,
            'status': 'error',
            'error': info.get('error', 'Setup failed'),
            'elapsed_sec': 0,
            'tokens': None,
            'model': alias
        })

# Display results
print(json.dumps(results, indent=2))

# Quick comparative view
print('\n' + '='*80)
print('COMPARISON SUMMARY')
print('='*80)
print(f"{'Alias':<20} {'Status':<15} {'Latency(s)':<15} {'Tokens':<15}")
print('-'*80)

for row in results:
    status = row.get('status', 'unknown')
    status_icon = '✅' if status == 'success' else '❌'
    latency_str = f"{row.get('elapsed_sec', 0):.3f}" if row.get('elapsed_sec') else 'N/A'
    
    # Handle token display - show if available or indicate estimated
    tokens = row.get('tokens')
    usage = row.get('usage', {})
    if tokens:
        if 'estimated_tokens' in usage:
            tokens_str = f"~{tokens} (est.)"
        else:
            tokens_str = str(tokens)
    else:
        tokens_str = 'N/A'
    
    print(f"{status_icon} {row['alias']:<18} {status:<15} {latency_str:<15} {tokens_str:<15}")

print('-'*80)

# Show detailed token breakdown if available
print("\nDetailed Token Usage:")
for row in results:
    if row.get('status') == 'success' and row.get('usage'):
        usage = row['usage']
        print(f"\n  {row['alias']}:")
        if 'prompt_tokens' in usage and usage['prompt_tokens']:
            print(f"    Prompt tokens:     {usage['prompt_tokens']}")
            print(f"    Completion tokens: {usage['completion_tokens']}")
            print(f"    Total tokens:      {usage['total_tokens']}")
        elif 'estimated_tokens' in usage:
            print(f"    Estimated prompt:     {usage['estimated_prompt_tokens']}")
            print(f"    Estimated completion: {usage['estimated_completion_tokens']}")
            print(f"    Estimated total:      {usage['estimated_tokens']}")
            print(f"    (API did not provide token counts - using ~4 chars/token estimate)")

print('\n' + '='*80)

# Calculate speedup if both succeeded
if len(results) == 2 and all(r.get('status') == 'success' and r.get('elapsed_sec') for r in results):
    speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec']
    print(f"\n💡 SLM is {speedup:.2f}x faster than LLM for this prompt")
    
    # Compare token throughput if available
    slm_tokens = results[0].get('tokens', 0)
    llm_tokens = results[1].get('tokens', 0)
    if slm_tokens and llm_tokens:
        slm_tps = slm_tokens / results[0]['elapsed_sec']
        llm_tps = llm_tokens / results[1]['elapsed_sec']
        print(f"   SLM throughput: {slm_tps:.1f} tokens/sec")
        print(f"   LLM throughput: {llm_tps:.1f} tokens/sec")
        
elif any(r.get('status') == 'error' for r in results):
    print(f"\n⚠️  Some models failed - check errors above")
    print("   Ensure Foundry Local is running: foundry service start")
    print("   Ensure models are loaded: foundry model run <model-name>")

results

[Init] Connecting to 'phi-4-mini' (attempt 1/2)...
[OK] Connected to 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[Init] Connecting to 'qwen2.5-7b' (attempt 1/2)...
[OK] Connected to 'qwen2.5-7b' -> qwen2.5-7b-instruct-cuda-gpu:3
     Endpoint: http://127.0.0.1:59959/v1
[
  {
    "alias": "phi-4-mini",
    "status": "success",
    "content": "1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the n

[{'alias': 'phi-4-mini',
  'status': 'success',
  'content': '1. Reduced Latency: Local AI inference can significantly reduce latency by processing data closer to the source, which is particularly beneficial for real-time applications such as autonomous vehicles or augmented reality.\n\n2. Enhanced Privacy: By keeping data processing local, sensitive information is less likely to be exposed to external networks, thereby enhancing privacy and security.\n\n3. Lower Bandwidth Usage: Local AI inference reduces the need for data transmission over the network, which can save bandwidth and reduce the risk of network congestion.\n\n4. Improved Reliability: Local processing can be more reliable, as it is less dependent on network connectivity. This is particularly important in scenarios where network connectivity is unreliable or intermittent.\n\n5. Scalability: Local AI inference can be easily scaled by adding more local processing units, making it easier to handle increasing data volumes or m

### Eredmények értelmezése

**Kulcsfontosságú mutatók:**
- **Késleltetés**: Minél alacsonyabb, annál jobb – gyorsabb válaszidőt jelez
- **Tokenek**: Nagyobb áteresztőképesség = több feldolgozott token
- **Útvonal**: Megerősíti, hogy melyik API végpontot használták

**Mikor használjuk az SLM-et vagy az LLM-et:**
- **SLM (Kis Nyelvi Modell)**: Gyors válaszok, alacsonyabb erőforrás-felhasználás, egyszerű feladatokhoz ideális
- **LLM (Nagy Nyelvi Modell)**: Magasabb minőség, jobb érvelés, akkor használjuk, amikor a minőség a legfontosabb

**Következő lépések:**
1. Próbálj ki különböző promptokat, hogy lásd, hogyan befolyásolja a komplexitás az összehasonlítást
2. Kísérletezz más modellpárokkal
3. Használd a Workshop router mintákat (06. szekció), hogy intelligensen irányítsd a feladatokat a komplexitás alapján


In [38]:
# Final Validation Check
print("="*70)
print("VALIDATION SUMMARY")
print("="*70)
print(f"✅ SLM Model: {SLM}")
print(f"✅ LLM Model: {LLM}")
print(f"✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager")
print(f"✅ Pre-flight passed: {all(v['status'] == 'success' for v in preflight.values()) if 'preflight' in dir() else 'Not run yet'}")
print(f"✅ Comparison completed: {len(results) == 2 if 'results' in dir() else 'Not run yet'}")
print(f"✅ Both models responded: {all(r.get('status') == 'success' for r in results) if 'results' in dir() and results else 'Not run yet'}")
print("="*70)

# Check for common configuration issues
issues = []
if 'LLM' in dir() and LLM not in ['qwen2.5-3b', 'qwen2.5-0.5b', 'qwen2.5-1.5b', 'qwen2.5-7b', 'phi-3.5-mini']:
    issues.append(f"⚠️  LLM is '{LLM}' - expected qwen2.5-3b for memory efficiency")
if 'preflight' in dir() and not all(v['status'] == 'success' for v in preflight.values()):
    issues.append("⚠️  Pre-flight check failed - models not accessible")
if 'results' in dir() and results and not all(r.get('status') == 'success' for r in results):
    issues.append("⚠️  Comparison incomplete - check for errors above")

if not issues and 'results' in dir() and results and all(r.get('status') == 'success' for r in results):
    print("🎉 ALL CHECKS PASSED! Notebook completed successfully.")
    print(f"   SLM ({SLM}) vs LLM ({LLM}) comparison completed.")
    if len(results) == 2:
        speedup = results[1]['elapsed_sec'] / results[0]['elapsed_sec'] if results[0]['elapsed_sec'] > 0 else 0
        print(f"   Performance: SLM is {speedup:.2f}x faster")
elif issues:
    print("\n⚠️  Issues detected:")
    for issue in issues:
        print(f"   {issue}")
    print("\n💡 Troubleshooting:")
    print("   1. Ensure service is running: foundry service start")
    print("   2. Load models: foundry model run phi-4-mini && foundry model run qwen2.5-7b")
    print("   3. Check model list: foundry model ls")
else:
    print("\n💡 Run all cells above first, then re-run this validation.")
print("="*70)

VALIDATION SUMMARY
✅ SLM Model: phi-4-mini
✅ LLM Model: qwen2.5-7b
✅ Using Foundry SDK Pattern: workshop_utils with FoundryLocalManager
✅ Pre-flight passed: True
✅ Comparison completed: True
✅ Both models responded: True
🎉 ALL CHECKS PASSED! Notebook completed successfully.
   SLM (phi-4-mini) vs LLM (qwen2.5-7b) comparison completed.
   Performance: SLM is 5.14x faster



---

**Felelősség kizárása**:  
Ezt a dokumentumot az [Co-op Translator](https://github.com/Azure/co-op-translator) AI fordítási szolgáltatás segítségével fordították le. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget a fordítás használatából eredő félreértésekért vagy téves értelmezésekért.
